# Obtener datos de especies

In [ ]:
from pyinaturalist import get_observations
import time
import csv

def get_species_by_place_id_adaptive():
    try:
        all_species = {}
        page = 1
        per_page = 100
        max_pages = 100  # Límite máximo de páginas para evitar bucles infinitos
        
        while page <= max_pages:
            print(f"Obteniendo página {page}...")
            
            try:
                response = get_observations(
                    place_id=8498,  # ID correcto de Nayarit
                    quality_grade='research',
                    per_page=per_page,
                    page=page
                )
                
                if not response or 'results' not in response or not response['results']:
                    print("No hay más resultados.")
                    break  # No hay más resultados
                    
                # Procesar observaciones de esta página
                page_species_count = 0
                for obs in response['results']:
                    if 'taxon' in obs and obs['taxon']:
                        taxon = obs['taxon']
                        taxon_id = taxon.get('id')
                        observation_id = obs.get('id')
                        
                        observation_url = f"https://www.inaturalist.org/observations/{observation_id}" if observation_id else "URL no disponible"
                        
                        if taxon_id and taxon_id not in all_species:
                            all_species[taxon_id] = {
                                'scientific_name': taxon.get('name', 'N/A'),
                                'common_name': taxon.get('preferred_common_name', 'No disponible'),
                                'rank': taxon.get('rank', 'N/A'),
                                'count': 0,
                                'observation_urls': []
                            }
                        
                        if taxon_id in all_species:
                            all_species[taxon_id]['count'] += 1
                            if observation_url not in all_species[taxon_id]['observation_urls']:
                                all_species[taxon_id]['observation_urls'].append(observation_url)
                        
                        page_species_count += 1
                
                print(f"Encontradas {page_species_count} observaciones en la página {page}")
                
                # Verificar si hay más páginas
                if len(response['results']) < per_page:
                    print("Última página alcanzada.")
                    break
                    
                page += 1
                time.sleep(0.5)  # Esperar medio segundo entre peticiones
                
            except Exception as e:
                if "Result window is too large" in str(e):
                    print(f"Límite de la API alcanzado en la página {page}. Continuando con los resultados obtenidos.")
                    break
                else:
                    # Si es otro tipo de error, lo relanzamos
                    raise e
        
        # Mostrar resultados
        print(f"\nResumen final: {len(all_species)} especies únicas encontradas en {page-1} páginas")
        print("=" * 80)
        
        # Ordenar por número de observaciones (descendente)
        sorted_species = sorted(all_species.items(), key=lambda x: x[1]['count'], reverse=True)
        
        # Mostrar solo las primeras 20 especies para no saturar la salida
        display_count = min(20, len(sorted_species))
        print(f"Mostrando las {display_count} especies más observadas:")
        
        for i, (taxon_id, species) in enumerate(sorted_species[:display_count], 1):
            print(f"{i:2d}. {species['scientific_name']:35} - {species['common_name']:25}")
            print(f"     Rango: {species['rank']:15} Observaciones: {species['count']:3d}")
            print(f"     URLs: {len(species['observation_urls'])} enlaces (ver CSV para lista completa)")
            print("-" * 60)
            
        # Guardar resultados completos en un archivo CSV
        save_to_csv(sorted_species)
        
    except Exception as e:
        print(f"Error: {e}")

def save_to_csv(species_list):
    """Guardar todos los resultados en un archivo CSV con todos los enlaces"""
    try:
        with open('especies_nayarit.csv', 'w', newline='', encoding='utf-8') as csvfile:
            fieldnames = [
                'numero', 
                'nombre_cientifico', 
                'nombre_comun', 
                'rango', 
                'cantidad_observaciones',
                'enlaces_observaciones'
            ]
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            
            writer.writeheader()
            
            for i, (taxon_id, species) in enumerate(species_list, 1):
                # Combinar todos los enlaces en una sola cadena separada por comas
                all_urls = ', '.join(species['observation_urls'])
                
                writer.writerow({
                    'numero': i,
                    'nombre_cientifico': species['scientific_name'],
                    'nombre_comun': species['common_name'],
                    'rango': species['rank'],
                    'cantidad_observaciones': species['count'],
                    'enlaces_observaciones': all_urls
                })
        
        print(f"\nResultados completos guardados en 'especies_nayarit.csv'")
        print(f"Total de especies: {len(species_list)}")
        
        # Mostrar estadísticas adicionales
        total_observaciones = sum(species['count'] for _, species in species_list)
        print(f"Total de observaciones: {total_observaciones}")
        
    except Exception as e:
        print(f"Error al guardar el archivo CSV: {e}")

get_species_by_place_id_adaptive()

Obteniendo página 1...
Encontradas 100 observaciones en la página 1
Obteniendo página 2...
Encontradas 100 observaciones en la página 2
Obteniendo página 3...
Encontradas 100 observaciones en la página 3
Obteniendo página 4...
Encontradas 100 observaciones en la página 4
Obteniendo página 5...
Encontradas 100 observaciones en la página 5
Obteniendo página 6...
Encontradas 100 observaciones en la página 6
Obteniendo página 7...
Encontradas 100 observaciones en la página 7
Obteniendo página 8...
Encontradas 100 observaciones en la página 8
Obteniendo página 9...
Encontradas 100 observaciones en la página 9
Obteniendo página 10...
Encontradas 100 observaciones en la página 10
Obteniendo página 11...
Encontradas 100 observaciones en la página 11
Obteniendo página 12...
Encontradas 100 observaciones en la página 12
Obteniendo página 13...
Encontradas 100 observaciones en la página 13
Obteniendo página 14...
Encontradas 100 observaciones en la página 14
Obteniendo página 15...
Encontradas 10

# Comprobar ruta común

In [3]:
from pyinaturalist import get_observations
from pygbif import species
import time
import csv

def obtener_todos_nombres_por_idioma(nombre_cientifico):
    """
    Obtiene todos los nombres comunes agrupados por idioma
    """
    try:
        resultados = species.name_lookup(
            q=nombre_cientifico,
            limit=1,
            verbose=True
        )
        
        if resultados['results']:
            primer_resultado = resultados['results'][0]
            
            nombres_por_idioma = {'spa': [], 'eng': [], 'otros': []}
            
            if 'vernacularNames' in primer_resultado and primer_resultado['vernacularNames']:
                for vname in primer_resultado['vernacularNames']:
                    language = vname.get('language', '')
                    vernacular_name = vname.get('vernacularName', '')
                    
                    if vernacular_name:
                        if language == 'spa':
                            if vernacular_name not in nombres_por_idioma['spa']:
                                nombres_por_idioma['spa'].append(vernacular_name)
                        elif language == 'eng':
                            if vernacular_name not in nombres_por_idioma['eng']:
                                nombres_por_idioma['eng'].append(vernacular_name)
                        else:
                            if vernacular_name not in nombres_por_idioma['otros']:
                                nombres_por_idioma['otros'].append(vernacular_name)
            
            return nombres_por_idioma
        else:
            return {"error": "Especie no encontrada"}
            
    except Exception as e:
        return {"error": f"Error: {e}"}

def formatear_nombres_comunes(nombre_cientifico):
    """
    Obtiene y formatea los nombres comunes en el orden requerido:
    español primero, luego inglés, luego otros idiomas
    """
    nombres_por_idioma = obtener_todos_nombres_por_idioma(nombre_cientifico)
    
    if 'error' in nombres_por_idioma:
        return "No disponible"
    
    # Construir la lista en el orden requerido
    lista_nombres = []
    
    # Español primero
    if nombres_por_idioma['spa']:
        lista_nombres.extend(nombres_por_idioma['spa'])
    
    # Inglés después
    if nombres_por_idioma['eng']:
        lista_nombres.extend(nombres_por_idioma['eng'])
    
    # Otros idiomas al final
    if nombres_por_idioma['otros']:
        lista_nombres.extend(nombres_por_idioma['otros'])
    
    # Si no hay nombres comunes, retornar "No disponible"
    if not lista_nombres:
        return "No disponible"
    
    # Convertir la lista a string separado por comas
    return ', '.join(lista_nombres)

def get_species_by_place_id_adaptive():
    try:
        all_species = {}
        page = 1
        per_page = 100
        max_pages = 100  # Límite máximo de páginas para evitar bucles infinitos
        
        while page <= max_pages:
            print(f"Obteniendo página {page}...")
            
            try:
                response = get_observations(
                    place_id=8498,  # ID correcto de Nayarit
                    quality_grade='research',
                    per_page=per_page,
                    page=page
                )
                
                if not response or 'results' not in response or not response['results']:
                    print("No hay más resultados.")
                    break  # No hay más resultados
                    
                # Procesar observaciones de esta página
                page_species_count = 0
                for obs in response['results']:
                    if 'taxon' in obs and obs['taxon']:
                        taxon = obs['taxon']
                        taxon_id = taxon.get('id')
                        observation_id = obs.get('id')
                        
                        observation_url = f"https://www.inaturalist.org/observations/{observation_id}" if observation_id else "URL no disponible"
                        
                        if taxon_id and taxon_id not in all_species:
                            scientific_name = taxon.get('name', 'N/A')
                            
                            # Obtener nombres comunes formateados usando GBIF
                            nombres_comunes_formateados = formatear_nombres_comunes(scientific_name)
                            
                            all_species[taxon_id] = {
                                'scientific_name': scientific_name,
                                'common_name': nombres_comunes_formateados,
                                'rank': taxon.get('rank', 'N/A'),
                                'count': 0,
                                'observation_urls': []
                            }
                        
                        if taxon_id in all_species:
                            all_species[taxon_id]['count'] += 1
                            if observation_url not in all_species[taxon_id]['observation_urls']:
                                all_species[taxon_id]['observation_urls'].append(observation_url)
                        
                        page_species_count += 1
                
                print(f"Encontradas {page_species_count} observaciones en la página {page}")
                
                # Verificar si hay más páginas
                if len(response['results']) < per_page:
                    print("Última página alcanzada.")
                    break
                    
                page += 1
                time.sleep(0.5)  # Esperar medio segundo entre peticiones
                
            except Exception as e:
                if "Result window is too large" in str(e):
                    print(f"Límite de la API alcanzado en la página {page}. Continuando con los resultados obtenidos.")
                    break
                else:
                    # Si es otro tipo de error, lo relanzamos
                    raise e
        
        # Mostrar resultados
        print(f"\nResumen final: {len(all_species)} especies únicas encontradas en {page-1} páginas")
        print("=" * 80)
        
        # Ordenar por número de observaciones (descendente)
        sorted_species = sorted(all_species.items(), key=lambda x: x[1]['count'], reverse=True)
        
        # Mostrar solo las primeras 20 especies para no saturar la salida
        display_count = min(20, len(sorted_species))
        print(f"Mostrando las {display_count} especies más observadas:")
        
        for i, (taxon_id, species) in enumerate(sorted_species[:display_count], 1):
            print(f"{i:2d}. {species['scientific_name']:35} - {species['common_name']:25}")
            print(f"     Rango: {species['rank']:15} Observaciones: {species['count']:3d}")
            print(f"     URLs: {len(species['observation_urls'])} enlaces (ver CSV para lista completa)")
            print("-" * 60)
            
        # Guardar resultados completos en un archivo CSV
        save_to_csv(sorted_species)
        
    except Exception as e:
        print(f"Error: {e}")

def save_to_csv(species_list):
    """Guardar todos los resultados en un archivo CSV con todos los enlaces"""
    try:
        with open('especies_nayarit.csv', 'w', newline='', encoding='utf-8') as csvfile:
            fieldnames = [
                'numero', 
                'nombre_cientifico', 
                'nombre_comun', 
                'rango', 
                'cantidad_observaciones',
                'enlaces_observaciones'
            ]
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            
            writer.writeheader()
            
            for i, (taxon_id, species) in enumerate(species_list, 1):
                # Combinar todos los enlaces en una sola cadena separada por comas
                all_urls = ', '.join(species['observation_urls'])
                
                writer.writerow({
                    'numero': i,
                    'nombre_cientifico': species['scientific_name'],
                    'nombre_comun': species['common_name'],
                    'rango': species['rank'],
                    'cantidad_observaciones': species['count'],
                    'enlaces_observaciones': all_urls
                })
        
        print(f"\nResultados completos guardados en 'especies_nayarit.csv'")
        print(f"Total de especies: {len(species_list)}")
        
        # Mostrar estadísticas adicionales
        total_observaciones = sum(species['count'] for _, species in species_list)
        print(f"Total de observaciones: {total_observaciones}")
        
    except Exception as e:
        print(f"Error al guardar el archivo CSV: {e}")

# Ejecutar la función principal
if __name__ == "__main__":
    get_species_by_place_id_adaptive()

INFO:Request:
GET https://api.inaturalist.org/v1/observations?quality_grade=research&place_id=8498&page=1&per_page=100
User-Agent: python-requests/2.32.5 pyinaturalist/0.20.2
Accept-Encoding: gzip, deflate
Accept: application/json
Connection: keep-alive



Obteniendo página 1...
Encontradas 100 observaciones en la página 1


INFO:Request:
GET https://api.inaturalist.org/v1/observations?quality_grade=research&place_id=8498&page=2&per_page=100
User-Agent: python-requests/2.32.5 pyinaturalist/0.20.2
Accept-Encoding: gzip, deflate
Accept: application/json
Connection: keep-alive



Obteniendo página 2...
Encontradas 100 observaciones en la página 2


INFO:Request:
GET https://api.inaturalist.org/v1/observations?quality_grade=research&place_id=8498&page=3&per_page=100
User-Agent: python-requests/2.32.5 pyinaturalist/0.20.2
Accept-Encoding: gzip, deflate
Accept: application/json
Connection: keep-alive



...

Obteniendo página 98...
Encontradas 100 observaciones en la página 98


INFO:Request:
GET https://api.inaturalist.org/v1/observations?quality_grade=research&place_id=8498&page=99&per_page=100
User-Agent: python-requests/2.32.5 pyinaturalist/0.20.2
Accept-Encoding: gzip, deflate
Accept: application/json
Connection: keep-alive



Obteniendo página 99...
Encontradas 100 observaciones en la página 99


INFO:Request:
GET https://api.inaturalist.org/v1/observations?quality_grade=research&place_id=8498&page=100&per_page=100
User-Agent: python-requests/2.32.5 pyinaturalist/0.20.2
Accept-Encoding: gzip, deflate
Accept: application/json
Connection: keep-alive



Obteniendo página 100...
Encontradas 100 observaciones en la página 100

Resumen final: 2057 especies únicas encontradas en 100 páginas
Mostrando las 20 especies más observadas:
 1. Ctenosaura pectinata                - Iguana Negra Mexicana, Iguana Mexicana de Cola Espinosa, Garrobos de Roca, Garrobo de Roca, Iguana-espinosa mexicana, Western Spiny-tailed Iguana, Mexican Black Spiny-tailed Iguana, Guerreran Spiny-tailed Iguana, Mexican spiny-tailed iguana, Spiny-tailed Iguana, Mexican Spiny-tailed Iguana, Mexican spinytail iguana, Iguane Mexicain à Queue Épineuse
     Rango: species         Observaciones: 219
     URLs: 219 enlaces (ver CSV para lista completa)
------------------------------------------------------------
 2. Kinosternon integrum                - No disponible            
     Rango: species         Observaciones: 140
     URLs: 140 enlaces (ver CSV para lista completa)
------------------------------------------------------------
 3. Iguana iguana                      

In [8]:
import pandas as pd
from urllib.parse import urlparse

def analizar_enlaces_comunes(archivo_csv):
    """
    Analiza la columna de enlaces_observaciones y encuentra la parte común que comparten TODOS los enlaces
    """
    try:
        # Leer el CSV
        df = pd.read_csv(archivo_csv)
        
        print(f"Analizando {len(df)} especies del archivo {archivo_csv}")
        print("=" * 60)
        
        # Obtener todos los enlaces únicos de todas las especies
        todos_enlaces = set()
        
        for index, row in df.iterrows():
            enlaces = str(row['enlaces_observaciones']).split(', ')
            todos_enlaces.update(enlaces)
        
        # Eliminar entradas vacías o inválidas
        todos_enlaces = {enlace for enlace in todos_enlaces if enlace.startswith('http')}
        
        print(f"Total de enlaces únicos encontrados: {len(todos_enlaces)}")
        
        if not todos_enlaces:
            print("No se encontraron enlaces válidos para analizar")
            return
        
        # Convertir a lista para procesar
        lista_enlaces = list(todos_enlaces)
        
        # Encontrar el prefijo común más largo
        prefijo_comun = encontrar_prefijo_comun(lista_enlaces)
        
        # Analizar componentes de URL
        analizar_componentes_url(lista_enlaces)
        
        return prefijo_comun
        
    except Exception as e:
        print(f"Error al analizar el archivo CSV: {e}")
        return None

def encontrar_prefijo_comun(enlaces):
    """
    Encuentra el prefijo común más largo que comparten TODOS los enlaces
    """
    if not enlaces:
        return ""
    
    # Tomar el primer enlace como referencia
    referencia = enlaces[0]
    prefijo_comun = referencia
    
    print("\nBuscando prefijo común...")
    print(f"Enlace de referencia: {referencia}")
    
    for enlace in enlaces[1:]:
        # Encontrar el prefijo común entre el prefijo actual y el siguiente enlace
        prefijo_temp = ""
        min_longitud = min(len(prefijo_comun), len(enlace))
        
        for i in range(min_longitud):
            if prefijo_comun[i] == enlace[i]:
                prefijo_temp += prefijo_comun[i]
            else:
                break
        
        prefijo_comun = prefijo_temp
        
        # Si ya no hay prefijo común, salir
        if not prefijo_comun:
            break
    
    print(f"Prefijo común encontrado: {prefijo_comun}")
    print(f"Longitud del prefijo común: {len(prefijo_comun)} caracteres")
    
    return prefijo_comun

def analizar_componentes_url(enlaces):
    """
    Analiza los componentes de URL para encontrar patrones comunes
    """
    print("\n" + "=" * 60)
    print("ANÁLISIS DETALLADO DE COMPONENTES DE URL")
    print("=" * 60)
    
    esquemas = set()
    netlocs = set()  # network location (dominio:puerto)
    paths_comunes = set()
    
    for enlace in enlaces:
        try:
            parsed = urlparse(enlace)
            esquemas.add(parsed.scheme)
            netlocs.add(parsed.netloc)
            
            # Extraer el path común (sin el ID específico)
            path = parsed.path
            if '/observations/' in path:
                # Tomar solo la parte hasta '/observations/'
                path_base = path.split('/observations/')[0] + '/observations/'
                paths_comunes.add(path_base)
            else:
                paths_comunes.add(path)
                
        except Exception as e:
            print(f"Error al parsear URL {enlace}: {e}")
    
    # Resultados del análisis
    print("\n1. ESQUEMAS (protocolos) utilizados:")
    for esquema in esquemas:
        print(f"   - {esquema}")
    
    print(f"\n2. DOMINIOS únicos encontrados ({len(netlocs)}):")
    for netloc in sorted(netlocs):
        print(f"   - {netloc}")
    
    print(f"\n3. PATHS base comunes ({len(paths_comunes)}):")
    for path in sorted(paths_comunes):
        print(f"   - {path}")
    
    # Verificar si todos comparten el mismo dominio
    if len(netlocs) == 1:
        dominio_comun = list(netlocs)[0]
        print(f"\n✓ TODOS los enlaces comparten el mismo dominio: {dominio_comun}")
    else:
        print(f"\n✗ Los enlaces utilizan {len(netlocs)} dominios diferentes")
    
    # Verificar si todos comparten el mismo esquema
    if len(esquemas) == 1:
        esquema_comun = list(esquemas)[0]
        print(f"✓ TODOS los enlaces utilizan el esquema: {esquema_comun}")
    else:
        print(f"✗ Los enlaces utilizan {len(esquemas)} esquemas diferentes")
    
    # Verificar si todos comparten el mismo path base
    if len(paths_comunes) == 1:
        path_comun = list(paths_comunes)[0]
        print(f"✓ TODOS los enlaces comparten el path base: {path_comun}")
    else:
        print(f"✗ Los enlaces utilizan {len(paths_comunes)} paths base diferentes")

def mostrar_estadisticas_enlaces(archivo_csv):
    """
    Muestra estadísticas generales sobre los enlaces
    """
    try:
        df = pd.read_csv(archivo_csv)
        
        print("\n" + "=" * 60)
        print("ESTADÍSTICAS GENERALES DE ENLACES")
        print("=" * 60)
        
        total_especies = len(df)
        total_enlaces = 0
        enlaces_por_especie = []
        
        for index, row in df.iterrows():
            enlaces = str(row['enlaces_observaciones']).split(', ')
            # Filtrar enlaces válidos
            enlaces_validos = [e for e in enlaces if e.startswith('http')]
            cantidad = len(enlaces_validos)
            enlaces_por_especie.append(cantidad)
            total_enlaces += cantidad
        
        print(f"Total de especies: {total_especies}")
        print(f"Total de enlaces a observaciones: {total_enlaces}")
        print(f"Promedio de enlaces por especie: {total_enlaces/total_especies:.2f}")
        print(f"Máximo de enlaces para una especie: {max(enlaces_por_especie)}")
        print(f"Mínimo de enlaces para una especie: {min(enlaces_por_especie)}")
        
        # Especies con más enlaces
        df['cantidad_enlaces'] = [len([e for e in str(row['enlaces_observaciones']).split(', ') 
                                     if e.startswith('http')]) 
                                for index, row in df.iterrows()]
        
        top_5 = df.nlargest(5, 'cantidad_enlaces')[['nombre_cientifico', 'cantidad_enlaces']]
        print(f"\nTop 5 especies con más observaciones:")
        for _, row in top_5.iterrows():
            print(f"  - {row['nombre_cientifico']}: {row['cantidad_enlaces']} enlaces")
            
    except Exception as e:
        print(f"Error al calcular estadísticas: {e}")

# Ejecutar el análisis
if __name__ == "__main__":
    archivo = 'especies_nayarit.csv'
    
    try:
        # Análisis principal
        prefijo_comun = analizar_enlaces_comunes(archivo)
        
        # Estadísticas adicionales
        mostrar_estadisticas_enlaces(archivo)
        
        print("\n" + "=" * 60)
        print("RESUMEN FINAL")
        print("=" * 60)
        if prefijo_comun:
            print(f"La parte común que COMPARTEN TODOS los enlaces es:")
            print(f"'{prefijo_comun}'")
        else:
            print("No se pudo determinar una parte común para todos los enlaces")
            
    except FileNotFoundError:
        print(f"Error: No se encontró el archivo {archivo}")
        print("Asegúrate de que el archivo CSV existe en el directorio actual")
    except Exception as e:
        print(f"Error inesperado: {e}")

Analizando 2057 especies del archivo especies_nayarit.csv
Total de enlaces únicos encontrados: 10000

Buscando prefijo común...
Enlace de referencia: https://www.inaturalist.org/observations/296903235
Prefijo común encontrado: https://www.inaturalist.org/observations/
Longitud del prefijo común: 41 caracteres

ANÁLISIS DETALLADO DE COMPONENTES DE URL

1. ESQUEMAS (protocolos) utilizados:
   - https

2. DOMINIOS únicos encontrados (1):
   - www.inaturalist.org

3. PATHS base comunes (1):
   - /observations/

✓ TODOS los enlaces comparten el mismo dominio: www.inaturalist.org
✓ TODOS los enlaces utilizan el esquema: https
✓ TODOS los enlaces comparten el path base: /observations/

ESTADÍSTICAS GENERALES DE ENLACES
Total de especies: 2057
Total de enlaces a observaciones: 10000
Promedio de enlaces por especie: 4.86
Máximo de enlaces para una especie: 219
Mínimo de enlaces para una especie: 1

Top 5 especies con más observaciones:
  - Ctenosaura pectinata: 219 enlaces
  - Kinosternon integ

# Formatear especies_nayarit.csv para que use el id después de /observations/

In [9]:
import pandas as pd
import re

def simplificar_csv_a_ids(archivo_csv):
    """
    Versión simple: reemplaza URLs por IDs en el archivo CSV original
    """
    try:
        # Leer el CSV
        df = pd.read_csv(archivo_csv)
        
        print(f"Procesando {len(df)} filas...")
        
        # Función para extraer IDs
        def extraer_ids(enlaces_str):
            if pd.isna(enlaces_str) or enlaces_str == '':
                return ''
            
            ids = []
            for enlace in str(enlaces_str).split(', '):
                # Buscar números después de /observations/
                match = re.search(r'/observations/(\d+)', enlace)
                if match:
                    ids.append(match.group(1))
            
            return ', '.join(ids)
        
        # Aplicar la función
        df['enlaces_observaciones'] = df['enlaces_observaciones'].apply(extraer_ids)
        
        # Guardar
        df.to_csv(archivo_csv, index=False, encoding='utf-8')
        
        print(f"✅ Completado: {archivo_csv} actualizado con IDs")
        
        # Mostrar ejemplo
        ejemplo = df[df['enlaces_observaciones'] != ''].iloc[0]
        print(f"📝 Ejemplo: {ejemplo['nombre_cientifico']}")
        print(f"   IDs: {ejemplo['enlaces_observaciones']}")
        
    except Exception as e:
        print(f"❌ Error: {e}")

# Uso rápido
if __name__ == "__main__":
    simplificar_csv_a_ids("especies_nayarit.csv")

Procesando 2057 filas...
✅ Completado: especies_nayarit.csv actualizado con IDs
📝 Ejemplo: Ctenosaura pectinata
   IDs: 328140817, 328076733, 327769738, 327623197, 327390359, 327203542, 326560191, 326536392, 326214189, 326214188, 326214183, 326214184, 326214178, 326214172, 325862806, 325741488, 325504029, 325228008, 325228007, 324978956, 324975702, 324408110, 323856624, 323856621, 323856543, 323856042, 323854309, 323834984, 323502969, 322686869, 322102339, 322078788, 321501174, 321305099, 321304632, 320984089, 320983850, 319992198, 319607659, 319598084, 319295563, 319210713, 319189280, 319188655, 319176230, 319062496, 319034285, 318466598, 318335006, 318080933, 317905792, 317905773, 317902576, 317313166, 317313072, 316752550, 316748750, 316438781, 315893429, 315724768, 315683510, 315400706, 314958689, 314958654, 314942790, 314914820, 314914814, 314914813, 314480866, 313619447, 313230883, 313230872, 312829156, 312828835, 312783307, 310208282, 310208274, 310208273, 308516183, 308238152, 

# Obtener coordenadas y crear un ubicaciones.csv con id = cada id de la columna enlace_observaciones de la columna del especies_nayarit.csv

## Conseguir coordenadas de una observación

In [11]:
from pyinaturalist import get_observations
import pandas as pd

def obtener_coordenadas_observacion(observation_id):
    """
    Obtiene las coordenadas de una observación específica
    """
    try:
        response = get_observations(id=observation_id)
        
        if response and 'results' in response and response['results']:
            observation = response['results'][0]
            
            # Extraer coordenadas
            lat = observation.get('latitude')
            lon = observation.get('longitude')
            location = observation.get('location')
            
            print(f"Observación {observation_id}:")
            print(f"  - Latitud: {lat}")
            print(f"  - Longitud: {lon}")
            print(f"  - Ubicación: {location}")
            
            # Información adicional útil
            print(f"  - Usuario: {observation.get('user', {}).get('login', 'N/A')}")
            print(f"  - Fecha: {observation.get('observed_on', 'N/A')}")
            print(f"  - Especie: {observation.get('taxon', {}).get('name', 'N/A')}")
            
            return {
                'latitude': lat,
                'longitude': lon,
                'location': location,
                'observation': observation
            }
        else:
            print(f"No se encontró la observación {observation_id}")
            return None
            
    except Exception as e:
        print(f"Error al obtener observación {observation_id}: {e}")
        return None

# Probar con tu ejemplo
observacion_id = 312829156
coordenadas = obtener_coordenadas_observacion(observacion_id)

Observación 312829156:
  - Latitud: None
  - Longitud: None
  - Ubicación: [21.4514572, -104.8987807]
  - Usuario: lucila-vilchez
  - Fecha: 2024-08-14 00:46:00-07:00
  - Especie: Ctenosaura pectinata


## Crear ubicaciones.csv

In [1]:
from pyinaturalist import get_observations
import pandas as pd
import time
import csv
import traceback
import os

def extraer_todos_los_ids(archivo_especies):
    """
    Extrae todos los IDs únicos de observaciones del archivo de especies
    """
    try:
        df = pd.read_csv(archivo_especies)
        todos_ids = set()
        
        print(f"Leyendo {len(df)} especies del archivo {archivo_especies}")
        
        for index, row in df.iterrows():
            ids_str = str(row['enlaces_observaciones'])
            if ids_str and ids_str != '' and ids_str != 'nan':
                # Dividir por comas y limpiar espacios
                ids_lista = [id_str.strip() for id_str in ids_str.split(',')]
                # Filtrar solo strings que sean números
                ids_numericos = [id_str for id_str in ids_lista if id_str.isdigit()]
                todos_ids.update(ids_numericos)
        
        print(f"✅ Total de IDs únicos de observación encontrados: {len(todos_ids)}")
        return list(todos_ids)
        
    except Exception as e:
        print(f"❌ Error al leer el archivo: {e}")
        print(traceback.format_exc())
        return []

def procesar_y_guardar_lotes(observation_ids, archivo_salida='ubicaciones.csv', batch_size=10, delay=1):
    """
    Procesa lotes de IDs y guarda los resultados inmediatamente en el CSV
    """
    try:
        # Verificar si el archivo ya existe para continuar desde donde se quedó
        ids_procesados = set()
        if os.path.exists(archivo_salida):
            df_existente = pd.read_csv(archivo_salida)
            ids_procesados = set(df_existente['id'].astype(str).tolist())
            print(f"📁 Archivo existente encontrado. {len(ids_procesados)} IDs ya procesados.")
        
        # Filtrar IDs pendientes
        ids_pendientes = [id_str for id_str in observation_ids if id_str not in ids_procesados]
        print(f"🔄 IDs pendientes por procesar: {len(ids_pendientes)}")
        
        if not ids_pendientes:
            print("✅ Todos los IDs ya han sido procesados.")
            return len(ids_procesados)
        
        # Abrir archivo en modo append si ya existe, o crear nuevo
        modo = 'a' if os.path.exists(archivo_salida) else 'w'
        with open(archivo_salida, modo, newline='', encoding='utf-8') as csvfile:
            fieldnames = ['id', 'latitud', 'longitud']
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            
            # Escribir encabezado solo si es archivo nuevo
            if modo == 'w':
                writer.writeheader()
            
            total_coordenadas_obtenidas = 0
            lotes_completados = 0
            
            # Procesar por lotes
            for i in range(0, len(ids_pendientes), batch_size):
                batch_ids = ids_pendientes[i:i + batch_size]
                batch_ids_int = [int(id_str) for id_str in batch_ids]
                
                print(f"\n🔄 Procesando lote {lotes_completados + 1} (IDs {i+1} a {min(i+batch_size, len(ids_pendientes))})...")
                
                try:
                    # Hacer la petición a la API
                    response = get_observations(id=batch_ids_int)
                    
                    if response and 'results' in response:
                        coordenadas_lote = 0
                        for observation in response['results']:
                            obs_id = str(observation['id'])
                            
                            # EXTRAER COORDENADAS DEL CAMPO LOCATION
                            location = observation.get('location')
                            lat = None
                            lon = None
                            
                            if location and isinstance(location, list) and len(location) >= 2:
                                # El campo location viene como [latitud, longitud]
                                lat = location[0]
                                lon = location[1]
                            elif observation.get('geojson') and observation['geojson'].get('coordinates'):
                                # Alternativa: usar geojson (viene como [longitud, latitud])
                                coordinates = observation['geojson']['coordinates']
                                if len(coordinates) >= 2:
                                    lon = coordinates[0]
                                    lat = coordinates[1]
                            
                            if lat is not None and lon is not None:
                                writer.writerow({
                                    'id': obs_id,
                                    'latitud': lat,
                                    'longitud': lon
                                })
                                coordenadas_lote += 1
                                total_coordenadas_obtenidas += 1
                                print(f"   ✅ {obs_id}: [{lat:.6f}, {lon:.6f}]")
                            else:
                                print(f"   ⚠️  {obs_id}: Sin coordenadas - "
                                      f"location: {location}, "
                                      f"geojson: {observation.get('geojson', {}).get('coordinates')}, "
                                      f"geoprivacy: {observation.get('geoprivacy')}")
                        
                        print(f"   ✅ Lote {lotes_completados + 1}: {coordenadas_lote}/{len(batch_ids)} coordenadas obtenidas")
                    
                    else:
                        print(f"   ❌ Lote {lotes_completados + 1}: Sin respuesta de la API")
                        # Mostrar detalles de la respuesta
                        if response:
                            print(f"      Respuesta: {response}")
                        else:
                            print(f"      No hubo respuesta")
                
                except Exception as e:
                    print(f"   ❌ Error en lote {lotes_completados + 1}:")
                    print(f"      Tipo de error: {type(e).__name__}")
                    print(f"      Mensaje: {str(e)}")
                    print(f"      Traceback completo:")
                    traceback.print_exc()
                    # Continuar con el siguiente lote incluso si este falla
                
                # Pausa entre lotes
                if i + batch_size < len(ids_pendientes):
                    print(f"   ⏳ Esperando {delay} segundo(s)...")
                    time.sleep(delay)
                
                lotes_completados += 1
        
        print(f"\n✅ Procesamiento completado:")
        print(f"   - Lotes procesados: {lotes_completados}")
        print(f"   - Coordenadas obtenidas en esta sesión: {total_coordenadas_obtenidas}")
        print(f"   - Total acumulado en el archivo: {len(ids_procesados) + total_coordenadas_obtenidas}")
        
        return len(ids_procesados) + total_coordenadas_obtenidas
        
    except Exception as e:
        print(f"❌ Error general en procesamiento por lotes:")
        print(f"   Tipo de error: {type(e).__name__}")
        print(f"   Mensaje: {str(e)}")
        traceback.print_exc()
        return 0

def crear_ubicaciones_csv_incremental(archivo_especies, archivo_salida='ubicaciones.csv', batch_size=10, delay=1):
    """
    Crea el archivo ubicaciones.csv procesando lotes incrementalmente
    """
    try:
        # Paso 1: Extraer todos los IDs
        todos_ids = extraer_todos_los_ids(archivo_especies)
        
        if not todos_ids:
            print("❌ No se encontraron IDs para procesar")
            return 0
        
        # Paso 2: Procesar por lotes y guardar incrementalmente
        registros_totales = procesar_y_guardar_lotes(todos_ids, archivo_salida, batch_size, delay)
        
        # Paso 3: Verificar resultado final
        if os.path.exists(archivo_salida):
            df_final = pd.read_csv(archivo_salida)
            print(f"\n📊 RESULTADO FINAL:")
            print(f"   - Registros en '{archivo_salida}': {len(df_final)}")
            print(f"   - IDs sin procesar: {len(todos_ids) - len(df_final)}")
            
            if len(df_final) > 0:
                print(f"\n📝 Primeros 5 registros:")
                print(df_final.head().to_string(index=False))
        
        return registros_totales
        
    except Exception as e:
        print(f"❌ Error en creación incremental:")
        traceback.print_exc()
        return 0

# Función para probar con un solo ID y verificar la respuesta
def probar_observacion_individual(observation_id):
    """
    Prueba obtener una observación individual para diagnosticar problemas
    """
    print(f"\n🔍 Probando observación individual: {observation_id}")
    try:
        response = get_observations(id=int(observation_id))
        
        print(f"   Respuesta completa recibida")
        
        if response and 'results' in response and response['results']:
            observation = response['results'][0]
            print(f"   ✅ Observación encontrada:")
            print(f"      - ID: {observation.get('id')}")
            print(f"      - Location: {observation.get('location')}")
            print(f"      - GeoJSON: {observation.get('geojson')}")
            print(f"      - ¿Coordenadas ocultas?: {observation.get('geoprivacy')}")
            print(f"      - Calidad: {observation.get('quality_grade')}")
            print(f"      - Especie: {observation.get('taxon', {}).get('name')}")
            
            # Extraer coordenadas del campo location
            location = observation.get('location')
            if location and isinstance(location, list) and len(location) >= 2:
                lat = location[0]
                lon = location[1]
                print(f"   ✅ COORDENADAS EXTRAÍDAS:")
                print(f"      - Latitud: {lat}")
                print(f"      - Longitud: {lon}")
                return True
            else:
                print(f"   ❌ No se pudieron extraer coordenadas del campo location")
                return False
        else:
            print(f"   ❌ No se encontró la observación {observation_id}")
            return False
            
    except Exception as e:
        print(f"   ❌ Error al obtener observación {observation_id}:")
        print(f"      Tipo: {type(e).__name__}")
        print(f"      Mensaje: {str(e)}")
        traceback.print_exc()
        return False

# Ejecutar el proceso completo
if __name__ == "__main__":
    archivo_especies = "especies_nayarit.csv"
    
    print("🚀 INICIANDO CREACIÓN INCREMENTAL DE UBICACIONES.CSV")
    print("=" * 60)
    
    # Primero, probar con un ID individual para diagnosticar
    print("🔍 FASE DE DIAGNÓSTICO:")
    id_prueba = "312829156"  # El mismo ID que usaste antes
    resultado_prueba = probar_observacion_individual(id_prueba)
    
    if resultado_prueba:
        print("\n" + "=" * 60)
        print("🎯 INICIANDO PROCESAMIENTO MASIVO")
        
        # Configuración de procesamiento
        batch_size = 5  # Lotes más pequeños para mejor control
        delay = 2       # Más delay entre lotes
        
        # Crear el archivo de ubicaciones
        registros = crear_ubicaciones_csv_incremental(
            archivo_especies, 
            batch_size=batch_size, 
            delay=delay
        )
        
        if registros > 0:
            print(f"\n🎉 ¡Proceso completado! {registros} registros en 'ubicaciones.csv'")
            
            # Verificar el archivo resultante
            if os.path.exists('ubicaciones.csv'):
                df_resultado = pd.read_csv('ubicaciones.csv')
                print(f"\n📊 VERIFICACIÓN DEL ARCHIVO:")
                print(f"   - Total de registros: {len(df_resultado)}")
                print(f"   - Ejemplo de coordenadas:")
                if len(df_resultado) > 0:
                    print(df_resultado.head(3).to_string(index=False))
        else:
            print("\n❌ No se pudieron obtener coordenadas.")
    else:
        print("\n❌ El diagnóstico falló. Revisa:")
        print("   - Tu conexión a internet")
        print("   - Que el ID de prueba sea válido")
        print("   - Que pyinaturalist esté correctamente instalado")

🚀 INICIANDO CREACIÓN INCREMENTAL DE UBICACIONES.CSV
🔍 FASE DE DIAGNÓSTICO:

🔍 Probando observación individual: 312829156
   Respuesta completa recibida
   ✅ Observación encontrada:
      - ID: 312829156
      - Location: [21.4514572, -104.8987807]
      - GeoJSON: {'type': 'Point', 'coordinates': [-104.8987807, 21.4514572]}
      - ¿Coordenadas ocultas?: None
      - Calidad: research
      - Especie: Ctenosaura pectinata
   ✅ COORDENADAS EXTRAÍDAS:
      - Latitud: 21.4514572
      - Longitud: -104.8987807

🎯 INICIANDO PROCESAMIENTO MASIVO
Leyendo 2057 especies del archivo especies_nayarit.csv
✅ Total de IDs únicos de observación encontrados: 10000
🔄 IDs pendientes por procesar: 10000

🔄 Procesando lote 1 (IDs 1 a 5)...
   ✅ 274076247: [21.081397, -105.147667]
   ✅ 290035743: [20.740937, -105.419245]
   ✅ 314934926: [20.811730, -105.466767]
   ✅ 315164886: [20.745769, -105.281497]
   ✅ 324187710: [21.513092, -104.920492]
   ✅ Lote 1: 5/5 coordenadas obtenidas
   ⏳ Esperando 2 segundo(

Traceback (most recent call last):
  File "/tmp/ipykernel_2078/2051035759.py", line 171, in crear_ubicaciones_csv_incremental
    df_final = pd.read_csv(archivo_salida)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/user/.local/lib/python3.11/site-packages/pandas/io/parsers/readers.py", line 1026, in read_csv
    return _read(filepath_or_buffer, kwds)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/user/.local/lib/python3.11/site-packages/pandas/io/parsers/readers.py", line 626, in _read
    return parser.read(nrows)
           ^^^^^^^^^^^^^^^^^^
  File "/home/user/.local/lib/python3.11/site-packages/pandas/io/parsers/readers.py", line 1923, in read
    ) = self._engine.read(  # type: ignore[attr-defined]
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/user/.local/lib/python3.11/site-packages/pandas/io/parsers/c_parser_wrapper.py", line 234, in read
    chunks = self._reader.read_low_memory(nrows)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

## Añadir fechas de la observación a "ubicaciones.csv"

### Fallido

In [ ]:
from pyinaturalist import get_observations
import pandas as pd
import time
import csv
import traceback
import os
from datetime import datetime

def extraer_todos_los_ids(archivo_especies):
    """
    Extrae todos los IDs únicos de observaciones del archivo de especies
    """
    try:
        df = pd.read_csv(archivo_especies)
        todos_ids = set()
        
        print(f"Leyendo {len(df)} especies del archivo {archivo_especies}")
        
        for index, row in df.iterrows():
            ids_str = str(row['enlaces_observaciones'])
            if ids_str and ids_str != '' and ids_str != 'nan':
                # Dividir por comas y limpiar espacios
                ids_lista = [id_str.strip() for id_str in ids_str.split(',')]
                # Filtrar solo strings que sean números
                ids_numericos = [id_str for id_str in ids_lista if id_str.isdigit()]
                todos_ids.update(ids_numericos)
        
        print(f"✅ Total de IDs únicos de observación encontrados: {len(todos_ids)}")
        return list(todos_ids)
        
    except Exception as e:
        print(f"❌ Error al leer el archivo: {e}")
        print(traceback.format_exc())
        return []

def procesar_y_guardar_lotes(observation_ids, archivo_salida='ubicaciones.csv', batch_size=10, delay=1):
    """
    Procesa lotes de IDs y guarda los resultados inmediatamente en el CSV
    """
    try:
        # Verificar si el archivo ya existe para continuar desde donde se quedó
        ids_procesados = set()
        if os.path.exists(archivo_salida):
            df_existente = pd.read_csv(archivo_salida)
            ids_procesados = set(df_existente['id'].astype(str).tolist())
            print(f"📁 Archivo existente encontrado. {len(ids_procesados)} IDs ya procesados.")
        
        # Filtrar IDs pendientes
        ids_pendientes = [id_str for id_str in observation_ids if id_str not in ids_procesados]
        print(f"🔄 IDs pendientes por procesar: {len(ids_pendientes)}")
        
        if not ids_pendientes:
            print("✅ Todos los IDs ya han sido procesados.")
            return len(ids_procesados)
        
        # Abrir archivo en modo append si ya existe, o crear nuevo
        modo = 'a' if os.path.exists(archivo_salida) else 'w'
        with open(archivo_salida, modo, newline='', encoding='utf-8') as csvfile:
            fieldnames = ['id', 'latitud', 'longitud', 'fecha']
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            
            # Escribir encabezado solo si es archivo nuevo
            if modo == 'w':
                writer.writeheader()
            
            total_coordenadas_obtenidas = 0
            lotes_completados = 0
            
            # Procesar por lotes
            for i in range(0, len(ids_pendientes), batch_size):
                batch_ids = ids_pendientes[i:i + batch_size]
                batch_ids_int = [int(id_str) for id_str in batch_ids]
                
                print(f"\n🔄 Procesando lote {lotes_completados + 1} (IDs {i+1} a {min(i+batch_size, len(ids_pendientes))})...")
                
                try:
                    # Hacer la petición a la API
                    response = get_observations(id=batch_ids_int)
                    
                    if response and 'results' in response:
                        coordenadas_lote = 0
                        for observation in response['results']:
                            obs_id = str(observation['id'])
                            
                            # EXTRAER COORDENADAS DEL CAMPO LOCATION
                            location = observation.get('location')
                            lat = None
                            lon = None
                            
                            if location and isinstance(location, list) and len(location) >= 2:
                                # El campo location viene como [latitud, longitud]
                                lat = location[0]
                                lon = location[1]
                            elif observation.get('geojson') and observation['geojson'].get('coordinates'):
                                # Alternativa: usar geojson (viene como [longitud, latitud])
                                coordinates = observation['geojson']['coordinates']
                                if len(coordinates) >= 2:
                                    lon = coordinates[0]
                                    lat = coordinates[1]
                            
                            # EXTRAER FECHA DE LA OBSERVACIÓN
                            fecha_observacion = None
                            observed_on = observation.get('observed_on')
                            
                            if observed_on:
                                # Si observed_on es un objeto datetime, convertirlo a string
                                if isinstance(observed_on, datetime):
                                    fecha_observacion = observed_on.strftime('%Y-%m-%d')
                                else:
                                    # Si es string, intentar parsearlo
                                    try:
                                        # Intentar varios formatos de fecha
                                        if 'T' in str(observed_on):
                                            fecha_obj = datetime.fromisoformat(observed_on.replace('Z', '+00:00'))
                                        else:
                                            fecha_obj = datetime.strptime(str(observed_on), '%Y-%m-%d')
                                        fecha_observacion = fecha_obj.strftime('%Y-%m-%d')
                                    except (ValueError, TypeError):
                                        # Si no se puede parsear, usar el string original
                                        fecha_observacion = str(observed_on)
                            else:
                                fecha_observacion = 'N/A'
                            
                            if lat is not None and lon is not None:
                                writer.writerow({
                                    'id': obs_id,
                                    'latitud': lat,
                                    'longitud': lon,
                                    'fecha': fecha_observacion
                                })
                                coordenadas_lote += 1
                                total_coordenadas_obtenidas += 1
                                print(f"   ✅ {obs_id}: [{lat:.6f}, {lon:.6f}] - {fecha_observacion}")
                            else:
                                print(f"   ⚠️  {obs_id}: Sin coordenadas - "
                                      f"location: {location}, "
                                      f"geojson: {observation.get('geojson', {}).get('coordinates')}, "
                                      f"fecha: {fecha_observacion}")
                        
                        print(f"   ✅ Lote {lotes_completados + 1}: {coordenadas_lote}/{len(batch_ids)} coordenadas obtenidas")
                    
                    else:
                        print(f"   ❌ Lote {lotes_completados + 1}: Sin respuesta de la API")
                
                except Exception as e:
                    print(f"   ❌ Error en lote {lotes_completados + 1}:")
                    print(f"      Tipo de error: {type(e).__name__}")
                    print(f"      Mensaje: {str(e)}")
                    traceback.print_exc()
                    # Continuar con el siguiente lote incluso si este falla
                
                # Pausa entre lotes
                if i + batch_size < len(ids_pendientes):
                    print(f"   ⏳ Esperando {delay} segundo(s)...")
                    time.sleep(delay)
                
                lotes_completados += 1
        
        print(f"\n✅ Procesamiento completado:")
        print(f"   - Lotes procesados: {lotes_completados}")
        print(f"   - Coordenadas obtenidas en esta sesión: {total_coordenadas_obtenidas}")
        print(f"   - Total acumulado en el archivo: {len(ids_procesados) + total_coordenadas_obtenidas}")
        
        return len(ids_procesados) + total_coordenadas_obtenidas
        
    except Exception as e:
        print(f"❌ Error general en procesamiento por lotes:")
        print(f"   Tipo de error: {type(e).__name__}")
        print(f"   Mensaje: {str(e)}")
        traceback.print_exc()
        return 0

def crear_ubicaciones_csv_incremental(archivo_especies, archivo_salida='ubicaciones.csv', batch_size=10, delay=1):
    """
    Crea el archivo ubicaciones.csv procesando lotes incrementalmente
    """
    try:
        # Paso 1: Extraer todos los IDs
        todos_ids = extraer_todos_los_ids(archivo_especies)
        
        if not todos_ids:
            print("❌ No se encontraron IDs para procesar")
            return 0
        
        # Paso 2: Procesar por lotes y guardar incrementalmente
        registros_totales = procesar_y_guardar_lotes(todos_ids, archivo_salida, batch_size, delay)
        
        # Paso 3: Verificar resultado final
        if os.path.exists(archivo_salida):
            df_final = pd.read_csv(archivo_salida)
            print(f"\n📊 RESULTADO FINAL:")
            print(f"   - Registros en '{archivo_salida}': {len(df_final)}")
            print(f"   - IDs sin procesar: {len(todos_ids) - len(df_final)}")
            
            if len(df_final) > 0:
                print(f"\n📝 Primeros 5 registros:")
                print(df_final.head().to_string(index=False))
                
                # Estadísticas de fechas
                fechas_validas = df_final[df_final['fecha'] != 'N/A']['fecha']
                if len(fechas_validas) > 0:
                    print(f"\n📅 ESTADÍSTICAS DE FECHAS:")
                    print(f"   - Rango de fechas: {fechas_validas.min()} a {fechas_validas.max()}")
                    print(f"   - Observaciones con fecha: {len(fechas_validas)}")
                    print(f"   - Observaciones sin fecha: {len(df_final) - len(fechas_validas)}")
        
        return registros_totales
        
    except Exception as e:
        print(f"❌ Error en creación incremental:")
        traceback.print_exc()
        return 0

# Función para probar con un solo ID y verificar la respuesta
def probar_observacion_individual(observation_id):
    """
    Prueba obtener una observación individual para diagnosticar problemas
    """
    print(f"\n🔍 Probando observación individual: {observation_id}")
    try:
        response = get_observations(id=int(observation_id))
        
        print(f"   Respuesta completa recibida")
        
        if response and 'results' in response and response['results']:
            observation = response['results'][0]
            print(f"   ✅ Observación encontrada:")
            print(f"      - ID: {observation.get('id')}")
            print(f"      - Location: {observation.get('location')}")
            print(f"      - Fecha (observed_on): {observation.get('observed_on')}")
            print(f"      - Fecha (observed_on_details): {observation.get('observed_on_details')}")
            print(f"      - ¿Coordenadas ocultas?: {observation.get('geoprivacy')}")
            print(f"      - Calidad: {observation.get('quality_grade')}")
            print(f"      - Especie: {observation.get('taxon', {}).get('name')}")
            
            # Extraer coordenadas del campo location
            location = observation.get('location')
            if location and isinstance(location, list) and len(location) >= 2:
                lat = location[0]
                lon = location[1]
                
                # Extraer fecha
                observed_on = observation.get('observed_on')
                fecha_formateada = 'N/A'
                if observed_on:
                    if isinstance(observed_on, datetime):
                        fecha_formateada = observed_on.strftime('%Y-%m-%d')
                    else:
                        try:
                            if 'T' in str(observed_on):
                                fecha_obj = datetime.fromisoformat(observed_on.replace('Z', '+00:00'))
                            else:
                                fecha_obj = datetime.strptime(str(observed_on), '%Y-%m-%d')
                            fecha_formateada = fecha_obj.strftime('%Y-%m-%d')
                        except (ValueError, TypeError):
                            fecha_formateada = str(observed_on)
                
                print(f"   ✅ DATOS EXTRAÍDOS:")
                print(f"      - Latitud: {lat}")
                print(f"      - Longitud: {lon}")
                print(f"      - Fecha: {fecha_formateada}")
                return True
            else:
                print(f"   ❌ No se pudieron extraer coordenadas del campo location")
                return False
        else:
            print(f"   ❌ No se encontró la observación {observation_id}")
            return False
            
    except Exception as e:
        print(f"   ❌ Error al obtener observación {observation_id}:")
        print(f"      Tipo: {type(e).__name__}")
        print(f"      Mensaje: {str(e)}")
        traceback.print_exc()
        return False

# Función para actualizar un archivo existente con la columna de fecha
def actualizar_csv_con_fecha(archivo_existente='ubicaciones.csv'):
    """
    Actualiza un CSV existente añadiendo la columna de fecha
    """
    try:
        if not os.path.exists(archivo_existente):
            print(f"❌ El archivo {archivo_existente} no existe")
            return
        
        # Leer el CSV existente
        df = pd.read_csv(archivo_existente)
        
        # Verificar si ya tiene la columna fecha
        if 'fecha' in df.columns:
            print(f"✅ El archivo ya tiene la columna 'fecha'")
            return
        
        print(f"📁 Actualizando {len(df)} registros en {archivo_existente}")
        
        # Añadir columna temporal para fecha
        df['fecha'] = 'N/A'
        
        # Procesar en lotes para obtener fechas
        batch_size = 10
        delay = 1
        
        for i in range(0, len(df), batch_size):
            batch_df = df.iloc[i:i + batch_size]
            batch_ids = batch_df['id'].astype(int).tolist()
            
            print(f"🔄 Procesando lote {i//batch_size + 1}...")
            
            try:
                response = get_observations(id=batch_ids)
                
                if response and 'results' in response:
                    for observation in response['results']:
                        obs_id = str(observation['id'])
                        observed_on = observation.get('observed_on')
                        
                        # Formatear fecha
                        fecha_formateada = 'N/A'
                        if observed_on:
                            if isinstance(observed_on, datetime):
                                fecha_formateada = observed_on.strftime('%Y-%m-%d')
                            else:
                                try:
                                    if 'T' in str(observed_on):
                                        fecha_obj = datetime.fromisoformat(observed_on.replace('Z', '+00:00'))
                                    else:
                                        fecha_obj = datetime.strptime(str(observed_on), '%Y-%m-%d')
                                    fecha_formateada = fecha_obj.strftime('%Y-%m-%d')
                                except (ValueError, TypeError):
                                    fecha_formateada = str(observed_on)
                        
                        # Actualizar el DataFrame
                        df.loc[df['id'] == int(obs_id), 'fecha'] = fecha_formateada
                
                time.sleep(delay)
                
            except Exception as e:
                print(f"❌ Error en lote {i//batch_size + 1}: {e}")
                continue
        
        # Guardar el archivo actualizado
        df.to_csv(archivo_existente, index=False)
        print(f"✅ Archivo actualizado con columna 'fecha'")
        
    except Exception as e:
        print(f"❌ Error al actualizar el archivo: {e}")

# Ejecutar el proceso completo
if __name__ == "__main__":
    archivo_especies = "especies_nayarit.csv"
    
    print("🚀 INICIANDO CREACIÓN DE UBICACIONES.CSV CON FECHAS")
    print("=" * 60)
    
    # Verificar si ya existe un archivo sin fechas
    if os.path.exists('ubicaciones.csv'):
        print("📁 Se encontró un archivo 'ubicaciones.csv' existente")
        respuesta = input("¿Deseas actualizarlo añadiendo la columna de fecha? (s/n): ").strip().lower()
        if respuesta == 's':
            actualizar_csv_con_fecha()
        else:
            print("✅ Continuando sin actualizar el archivo existente")
    
    # Primero, probar con un ID individual para diagnosticar
    print("\n🔍 FASE DE DIAGNÓSTICO:")
    id_prueba = "312829156"
    resultado_prueba = probar_observacion_individual(id_prueba)
    
    if resultado_prueba:
        print("\n" + "=" * 60)
        print("🎯 INICIANDO PROCESAMIENTO MASIVO")
        
        # Configuración de procesamiento
        batch_size = 5
        delay = 2
        
        # Crear el archivo de ubicaciones con fechas
        registros = crear_ubicaciones_csv_incremental(
            archivo_especies, 
            batch_size=batch_size, 
            delay=delay
        )
        
        if registros > 0:
            print(f"\n🎉 ¡Proceso completado! {registros} registros en 'ubicaciones.csv'")
        else:
            print("\n❌ No se pudieron obtener coordenadas.")
    else:
        print("\n❌ El diagnóstico falló.")

🚀 INICIANDO CREACIÓN DE UBICACIONES.CSV CON FECHAS
📁 Se encontró un archivo 'ubicaciones.csv' existente


📁 Actualizando 9999 registros en ubicaciones.csv
🔄 Procesando lote 1...
🔄 Procesando lote 2...
🔄 Procesando lote 3...
🔄 Procesando lote 4...
🔄 Procesando lote 5...
🔄 Procesando lote 6...
🔄 Procesando lote 7...
🔄 Procesando lote 8...
🔄 Procesando lote 9...
🔄 Procesando lote 10...
🔄 Procesando lote 11...
🔄 Procesando lote 12...
🔄 Procesando lote 13...
🔄 Procesando lote 14...
🔄 Procesando lote 15...
🔄 Procesando lote 16...
🔄 Procesando lote 17...
🔄 Procesando lote 18...
🔄 Procesando lote 19...
🔄 Procesando lote 20...
🔄 Procesando lote 21...
🔄 Procesando lote 22...
🔄 Procesando lote 23...
🔄 Procesando lote 24...
🔄 Procesando lote 25...
🔄 Procesando lote 26...
🔄 Procesando lote 27...
🔄 Procesando lote 28...
🔄 Procesando lote 29...
🔄 Procesando lote 30...
🔄 Procesando lote 31...
🔄 Procesando lote 32...
🔄 Procesando lote 33...
🔄 Procesando lote 34...
🔄 Procesando lote 35...
🔄 Procesando lote 36...
🔄 Procesando lote 37...
🔄 Procesando lote 38...
🔄 Procesando lote 39...
🔄 Procesando lote 40...


### Nuevo

In [ ]:
import pandas as pd
import pyinaturalist
import time
from datetime import datetime
import os

def obtener_fecha_observacion(observation_id):
    """Obtiene la fecha de una observación usando pyinaturalist"""
    try:
        response = pyinaturalist.get_observation(observation_id)
        observed_date = response['observed_on']
        
        if observed_date:
            return observed_date.strftime('%Y-%m-%d')
        else:
            return None
    except Exception as e:
        print(f"Error al obtener fecha para observación {observation_id}: {e}")
        return None

def procesar_observaciones(archivo_csv):
    """Procesa el CSV y agrega las fechas de las observaciones"""
    
    # Leer el archivo CSV
    df = pd.read_csv(archivo_csv)
    
    # Verificar si la columna 'fecha' ya existe, si no, crearla
    if 'fecha' not in df.columns:
        df['fecha'] = None
    
    # Contador para mostrar progreso
    total = len(df)
    procesadas = 0
    
    # Procesar cada fila
    for index, row in df.iterrows():
        # Si ya tiene fecha, saltar
        if pd.notna(df.at[index, 'fecha']):
            procesadas += 1
            continue
            
        observation_id = row['id']
        
        print(f"Procesando {procesadas + 1}/{total}: ID {observation_id}")
        
        # Obtener la fecha
        fecha = obtener_fecha_observacion(observation_id)
        
        # Actualizar el DataFrame
        df.at[index, 'fecha'] = fecha
        
        # Guardar inmediatamente en el CSV
        df.to_csv(archivo_csv, index=False)
        
        procesadas += 1
        
        # Pequeña pausa para no saturar la API (respetuoso con el servidor)
        time.sleep(1)
    
    print("Proceso completado!")
    return df

# Ejecutar el programa
if __name__ == "__main__":
    archivo = "ubicaciones.csv"
    
    # Verificar si el archivo existe
    if not os.path.exists(archivo):
        print(f"Error: El archivo {archivo} no existe")
    else:
        print(f"Iniciando procesamiento de {archivo}")
        df_resultado = procesar_observaciones(archivo)
        print("Resumen del resultado:")
        print(df_resultado.head())
        print(f"\nTotal de observaciones procesadas: {len(df_resultado)}")

Iniciando procesamiento de ubicaciones.csv
Procesando 1/9999: ID 274076247
Procesando 2/9999: ID 290035743
Procesando 3/9999: ID 314934926
Procesando 4/9999: ID 315164886
Procesando 5/9999: ID 324187710
Procesando 6/9999: ID 297440432
Procesando 7/9999: ID 298548956
Procesando 8/9999: ID 306091556
Procesando 9/9999: ID 319426586
Procesando 10/9999: ID 323537864
Procesando 11/9999: ID 269363096
Procesando 12/9999: ID 282710731
Procesando 13/9999: ID 300209866
Procesando 14/9999: ID 313973662
Procesando 15/9999: ID 326760086
Procesando 16/9999: ID 283024915
Procesando 17/9999: ID 292875494
Procesando 18/9999: ID 292991798
Procesando 19/9999: ID 293241309
Procesando 20/9999: ID 305586344
Procesando 21/9999: ID 270097282
Procesando 22/9999: ID 271689566
Procesando 23/9999: ID 309341082
Procesando 24/9999: ID 319588761
Procesando 25/9999: ID 319782814
Procesando 26/9999: ID 269525785
Procesando 27/9999: ID 277091428
Procesando 28/9999: ID 286074090
Procesando 29/9999: ID 292694414
Procesand

In [ ]:
import pandas as pd
import pyinaturalist
import time
from datetime import datetime
import os

def obtener_fecha_observacion(observation_id):
    """Obtiene la fecha de una observación usando pyinaturalist"""
    try:
        response = pyinaturalist.get_observation(observation_id)
        observed_date = response['observed_on']
        
        if observed_date:
            return observed_date.strftime('%Y-%m-%d')
        else:
            return None
    except Exception as e:
        print(f"Error al obtener fecha para observación {observation_id}: {e}")
        return None

def procesar_observaciones_desde_linea(archivo_csv, linea_inicio=0):
    """Procesa el CSV y agrega las fechas de las observaciones comenzando desde una línea específica"""
    
    # Leer el archivo CSV
    df = pd.read_csv(archivo_csv)
    
    # Verificar si la columna 'fecha' ya existe, si no, crearla
    if 'fecha' not in df.columns:
        df['fecha'] = None
    
    # Contador para mostrar progreso
    total = len(df)
    
    # Asegurarnos de que la línea de inicio sea válida
    linea_inicio = max(0, min(linea_inicio, total - 1))
    
    print(f"Iniciando desde la línea {linea_inicio + 1} de {total}")
    
    # Procesar cada fila comenzando desde linea_inicio
    for index in range(linea_inicio, total):
        # Si ya tiene fecha, saltar
        if pd.notna(df.at[index, 'fecha']):
            print(f"Línea {index + 1}/{total}: ID {df.at[index, 'id']} ya tiene fecha, saltando...")
            continue
            
        observation_id = df.at[index, 'id']
        
        print(f"Procesando {index + 1}/{total}: ID {observation_id}")
        
        # Obtener la fecha
        fecha = obtener_fecha_observacion(observation_id)
        
        # Actualizar el DataFrame
        df.at[index, 'fecha'] = fecha
        
        # Guardar inmediatamente en el CSV
        df.to_csv(archivo_csv, index=False)
        
        # Pequeña pausa para no saturar la API (respetuoso con el servidor)
        time.sleep(1)
    
    print("Proceso completado!")
    return df

# Ejecutar el programa
if __name__ == "__main__":
    archivo = "ubicaciones.csv"
    
    # Verificar si el archivo existe
    if not os.path.exists(archivo):
        print(f"Error: El archivo {archivo} no existe")
    else:
        print(f"Iniciando procesamiento de {archivo}")
        
        linea_inicio = 1995
        
        df_resultado = procesar_observaciones_desde_linea(archivo, linea_inicio)
        
        print("Resumen del resultado:")
        print(df_resultado.tail(10))  # Mostrar las últimas 10 filas
        print(f"\nTotal de observaciones procesadas: {len(df_resultado)}")
        print(f"Observaciones procesadas en esta ejecución: {len(df_resultado) - linea_inicio}")

Iniciando procesamiento de ubicaciones.csv
Iniciando desde la línea 1996 de 9999
Procesando 1996/9999: ID 272615137
Procesando 1997/9999: ID 297467534
Procesando 1998/9999: ID 308238152
Procesando 1999/9999: ID 325782391
Procesando 2000/9999: ID 327673486
Procesando 2001/9999: ID 298548960
Procesando 2002/9999: ID 299918340
Procesando 2003/9999: ID 302486403
Procesando 2004/9999: ID 309079083
Procesando 2005/9999: ID 325781780
Procesando 2006/9999: ID 270651435
Procesando 2007/9999: ID 281873289
Procesando 2008/9999: ID 292875198
Procesando 2009/9999: ID 301705853
Procesando 2010/9999: ID 305582908
Procesando 2011/9999: ID 278579587
Procesando 2012/9999: ID 300819281
Procesando 2013/9999: ID 305572037
Procesando 2014/9999: ID 315813022
Procesando 2015/9999: ID 326095733
Procesando 2016/9999: ID 271323260
Procesando 2017/9999: ID 298985908
Procesando 2018/9999: ID 303068663
Procesando 2019/9999: ID 304060288
Procesando 2020/9999: ID 318454549
Procesando 2021/9999: ID 270530783
Procesand

In [4]:
import pyinaturalist
from datetime import datetime

# ID de la observación (extraído de la URL)
observation_id = 291528754

# Obtener los datos de la observación
response = pyinaturalist.get_observation(observation_id)

# Extraer la fecha de la observación directamente del response
observed_date = response['observed_on']

# Formatear la fecha si está disponible
if observed_date:
    # Convertir el objeto datetime a string formateado
    formatted_date = observed_date.strftime('%d/%m/%Y')
    print(f'Fecha de la observación: {formatted_date}')
else:
    print('La observación no tiene fecha registrada')

Fecha de la observación: 21/06/2025
